In [1]:
from model.BPRMF import *
from preprocess.AmazonBook import *
from evaluation.BPRMF_evaluation import *
from evaluation.LightGCN_SISA import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/amazon-book'
dataset = AmazonBook(path)

data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'lr': 1e-4,
    'epochs': 120,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
}
model = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)

# Split the dataset

In [2]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

# Retain BPRMF model

In [3]:
config['epochs'] = 120
retrain_lightgcn = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)
retrain_lightgcn, epoch_tracks, test_topks = BPRMF_eva(retrain_lightgcn, config, retain_data, device)

Epoch: 001, Loss: 0.6949, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 002, Loss: 0.6937, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 003, Loss: 0.6926, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 004, Loss: 0.6914, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 005, Loss: 0.6903, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 006, Loss: 0.6892, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 007, Loss: 0.6880, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 008, Loss: 0.6869, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 009, Loss: 0.6856, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 010, Loss: 0.6845, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 011, Loss: 0.6833, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 012, Loss: 0.6821, Precision@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoc

In [4]:
# Test on the forget data
BPRMF_forget_data_eva(retrain_lightgcn, None, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0075, Recall@20: 0.0373, NDCG@20: 0.0207


# Prompt Unlearning
## Case 1: Without Contrastive Loss and Regularization

In [5]:
# Define the model
teacher = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)
student = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)

# Load the model
teacher.load_state_dict(torch.load(f"BPRMF_Amazon_Book_{config['epochs']}_Epochs_Top_{config['k']}.pt"))
student.load_state_dict(torch.load(f"BPRMF_Amazon_Book_{config['epochs']}_Epochs_Top_{config['k']}.pt"))

<All keys matched successfully>

In [7]:
# Setting the basic hyperparameters
config['beta'] = 0.6
config['alpha'] = 0.4
config['epochs'] = 50
config['gamma'] = 1e-6 # contrastive loss
config['delta'] = 1e-2 # regularization loss
config['tuning_type'] = 'gpf'
config['weight_decay'] = 0.001
config['regularization'] = False
config['Contrastive_loss'] = False
student, prompt, epoch_tracks, test_topks = prompt_BPRMF_unlearning_eva(teacher, student, retain_data, forget_data, config, device)

c:\Users\MSI\.conda\envs\master\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'x', 'edge_label_index', 'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Epoch: 001, Loss: 0.3679, HR@20: 0.0084, Recall@20: 0.0185, NDCG@20: 0.0143
Epoch: 002, Loss: 0.3688, HR@20: 0.0084, Recall@20: 0.0185, NDCG@20: 0.0143
Epoch: 003, Loss: 0.3683, HR@20: 0.0084, Recall@20: 0.0185, NDCG@20: 0.0143
Epoch: 004, Loss: 0.3574, HR@20: 0.0083, Recall@20: 0.0184, NDCG@20: 0.0142
Epoch: 005, Loss: 0.3292, HR@20: 0.0083, Recall@20: 0.0184, NDCG@20: 0.0142
Epoch: 006, Loss: 0.3364, HR@20: 0.0083, Recall@20: 0.0183, NDCG@20: 0.0142
Epoch: 007, Loss: 0.3279, HR@20: 0.0083, Recall@20: 0.0183, NDCG@20: 0.0141
Epoch: 008, Loss: 0.3268, HR@20: 0.0083, Recall@20: 0.0182, NDCG@20: 0.0141
Epoch: 009, Loss: 0.3124, HR@20: 0.0083, Recall@20: 0.0182, NDCG@20: 0.0141
Epoch: 010, Loss: 0.3132, HR@20: 0.0083, Recall@20: 0.0182, NDCG@20: 0.0141
Epoch: 011, Loss: 0.3168, HR@20: 0.0083, Recall@20: 0.0181, NDCG@20: 0.0140
Epoch: 012, Loss: 0.3045, HR@20: 0.0083, Recall@20: 0.0181, NDCG@20: 0.0140
Epoch: 013, Loss: 0.3093, HR@20: 0.0082, Recall@20: 0.0181, NDCG@20: 0.0140
Epoch: 014, 

In [8]:
# Test on the forget data
BPRMF_forget_data_eva(student, prompt, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0128, Recall@20: 0.0684, NDCG@20: 0.0398


In [9]:
config['regularization'] = False
config['Contrastive_loss'] = True
student, prompt, epoch_tracks, test_topks = prompt_BPRMF_unlearning_eva(teacher, student, retain_data, forget_data, config, device)

c:\Users\MSI\.conda\envs\master\lib\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'x', 'edge_label_index', 'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Epoch: 001, Loss: 0.6002, HR@20: 0.0079, Recall@20: 0.0173, NDCG@20: 0.0136
Epoch: 002, Loss: 0.5944, HR@20: 0.0079, Recall@20: 0.0173, NDCG@20: 0.0136
Epoch: 003, Loss: 0.6000, HR@20: 0.0079, Recall@20: 0.0174, NDCG@20: 0.0136
Epoch: 004, Loss: 0.5936, HR@20: 0.0079, Recall@20: 0.0174, NDCG@20: 0.0137
Epoch: 005, Loss: 0.5973, HR@20: 0.0080, Recall@20: 0.0174, NDCG@20: 0.0137
Epoch: 006, Loss: 0.5829, HR@20: 0.0080, Recall@20: 0.0175, NDCG@20: 0.0137
Epoch: 007, Loss: 0.5908, HR@20: 0.0080, Recall@20: 0.0175, NDCG@20: 0.0137
Epoch: 008, Loss: 0.5888, HR@20: 0.0080, Recall@20: 0.0175, NDCG@20: 0.0138
Epoch: 009, Loss: 0.5833, HR@20: 0.0080, Recall@20: 0.0176, NDCG@20: 0.0138
Epoch: 010, Loss: 0.5856, HR@20: 0.0080, Recall@20: 0.0176, NDCG@20: 0.0138
Epoch: 011, Loss: 0.5724, HR@20: 0.0080, Recall@20: 0.0176, NDCG@20: 0.0138
Epoch: 012, Loss: 0.5779, HR@20: 0.0080, Recall@20: 0.0176, NDCG@20: 0.0138
Epoch: 013, Loss: 0.5765, HR@20: 0.0081, Recall@20: 0.0177, NDCG@20: 0.0139
Epoch: 014, 

In [10]:
# Test on the forget data
BPRMF_forget_data_eva(student, prompt, forget_data, config['num_users'], config['k'], config['batch_size'], device)

HR@20: 0.0129, Recall@20: 0.0707, NDCG@20: 0.0405
